# PFRA 

### _Development Notebook_

__NOTE__ *Some settings are currently in place for PFRA Studies conducted by STARR II production studies. With time these will be transitioned to __general case options__.*

   - Assumes outputs written to zip files with '_out.zip' extensions
   - Assumes only one '.hdf' file in the out_zip
   - Assumes PFRA data structure with data stored on S3 with AWS credentials configured
   
   
--- 

### QA/QC
#### Global Errors Check

This notebook is intended to identify global issues in a model simulation that require follow-up responses from the modeler before the model can be used in *STARRII* PFRA studies.

Global issues currently implemented include:
 
 __1. Check Model inputs__
 
    A. Plan Information
    B. Plan Parameters
    C. Boundary Conditions 
     
__2. Check Results for each domain (2D Flow Area) in the model__
 
    A. Computational Summary & Depth Plots
    B. Domain Extents
    C. High Velocities at cell faces
 
 
Global issues *in development* include:
 - Mapping areas where Courant number exceeds indicated threshold.
 - Mapping areas where iteration counts exceed indicated threshold.


In [ ]:
import sys;sys.path.append(r'C:\Users\Administrator\Desktop\hecrasio')
from hecrasio.core import *
from hecrasio.qaqc import *
#import rasterio
import time

#### Read model output data 

*from AWS*




In [ ]:
model = ResultsZip(model_s3path, require_prj=False, pfra=False)

In [ ]:
hdfResults_paths = [f for f in model.contents if '.hdf' in f]

In [ ]:
hdfResults_path = hdfResults_paths[0]
rasPlan = HDFResultsFile(model, hdfResults_path)


### 1A. Plan Information

   

In [ ]:
rasPlan.Plan_Information

### 1B. Plan Parameters


In [ ]:
rasPlan.Plan_Parameters

### 1C. Boundary Conditions 

#### Input hydrographs

* Need to update notebook for multiple domains

In [ ]:
show_results(rasPlan.domains, model, rasPlan)

### 2A. Computational Summary



In [ ]:
rasPlan.summary

### 3A. Raster values at points
#### Confirm points are in same projection

In [ ]:
#pnts = gpd.read_file(points)
#tif = rasterio.open(tiffs[0])

#print("The points have {} coordinate reference system while the GeoTIFF is {}.".format(pnts.crs, tif.crs))

In [ ]:
#pnts = PointData(points).geodataframe

In [ ]:
#start = time.time()
#df = extract_values_at_points(pnts, tiffs[0:30])
#end = round(time.time()-start,2)
#print("Identifying raster values at points took {} seconds".format(end))

# END